# Train and Evaluate Models

In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
from collections import Counter

In [3]:
from sklearn.compose import ColumnTransformer

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [5]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import VarianceThreshold

In [6]:
from sklearn.utils import compute_sample_weight
from sklearn.utils import class_weight

In [7]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
from sklearn.metrics import classification_report

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [9]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
from sklearn.metrics import classification_report

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

In [11]:
# Models
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

In [14]:
# Set display and format settings

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.0f}'.format

In [15]:
DATA_ROOT_DIR='/mnt/data/projects/MD7'
PROJ_ROOT_DIR='/home/priyesh/projects/MD7'

### Load Data

In [16]:
filepath=os.path.join(PROJ_ROOT_DIR,'pickle','yahoo_complete_flat.pkl')
df_complete_flat = pd.read_pickle(filepath)

In [17]:
filepath=os.path.join(PROJ_ROOT_DIR,'pickle','yahoo_complete.pkl')
df_complete = pd.read_pickle(filepath)

In [18]:
# Take a copy so that we can always refer back to original 

df = df_complete_flat.copy()

In [19]:
# Rename final score to label

df.rename(columns={'FinalScore':'label'},inplace=True)

In [20]:
# Drop column company

df.drop('company',axis=1,inplace=True)

In [21]:
df .head()

,industry,0_inc_totalRevenue,0_inc_costOfRevenue,0_inc_grossProfit,0_inc_netIncome,0_inc_operatingIncome,0_inc_netIncomeContinuousOperations,0_inc_netInterestIncome,0_inc_interestIncome,0_inc_otherIncomeExpense,0_inc_operatingExpense,0_inc_totalExpenses,0_inc_taxProvision,0_inc_interestExpense,0_inc_SGA,0_inc_researchAndDevelopment,0_inc_ebit,0_inc_dilutedEPS,0_inc_basicEPS,0_bal_cashEquivalent,0_bal_receivables,0_bal_finishedGoods,0_bal_workInProcess,0_bal_rawMaterials,0_bal_otherCurrentAssets,0_bal_inventory,0_bal_currentAssets,0_bal_netPPE,0_bal_otherNonCurrentAssets,0_bal_financialAssets,0_bal_goodwill,0_bal_goodwillAndOtherIntangibleAssets,0_bal_otherIntangibleAssets,0_bal_nonCurrentAccountsReceivable,0_bal_totalNonCurrentAssets,0_bal_currentDebt,0_bal_payablesAndAccruedExpenses,0_bal_otherCurrentLiabilities,0_bal_currentLiabilities,0_bal_longTermDebt,0_bal_otherNonCurrentLiabilities,0_bal_nonCurrentDeferredLiabilities,0_bal_nonCurrentDeferredTaxesLiabilities,0_bal_longTermProvisions,0_bal_totalNonCurrentLiabilities,0_bal_retainedEarnings,0_bal_stockholdersEquity,0_bal_totalDebt,0_bal_totalAssets,0_bal_totalLiabilities,0_bal_workingCapital,0_bal_quickRatio,0_bal_currentRatio,0_bal_debtCapitalRatio,0_bal_debtAssetRatio,0_cas_depreciation,0_cas_stockBasedCompensation,0_cas_cashFlowOperatingActivities,0_cas_capitalExpenditure,0_cas_saleOfBusiness,0_cas_purchaseOfPPE,0_cas_cashFlowInvestingActivities,0_cas_cashDividendsPaid,0_cas_repurchaseOfCapitalStock,0_cas_cashFlowFinancingActivities,0_cas_freeCashFlow,1_inc_totalRevenue,1_inc_costOfRevenue,1_inc_grossProfit,1_inc_netIncome,1_inc_operatingIncome,1_inc_netIncomeContinuousOperations,1_inc_netInterestIncome,1_inc_interestIncome,1_inc_otherIncomeExpense,1_inc_operatingExpense,1_inc_totalExpenses,1_inc_taxProvision,1_inc_interestExpense,1_inc_SGA,1_inc_researchAndDevelopment,1_inc_ebit,1_inc_dilutedEPS,1_inc_basicEPS,1_bal_cashEquivalent,1_bal_receivables,1_bal_finishedGoods,1_bal_workInProcess,1_bal_rawMaterials,1_bal_otherCurrentAssets,1_bal_inventory,1_bal_currentAssets,1_bal_netPPE,1_bal_otherNonCurrentAssets,1_bal_financialAssets,1_bal_goodwill,1_bal_goodwillAndOtherIntangibleAssets,1_bal_otherIntangibleAssets,1_bal_nonCurrentAccountsReceivable,1_bal_totalNonCurrentAssets,1_bal_currentDebt,1_bal_payablesAndAccruedExpenses,1_bal_otherCurrentLiabilities,1_bal_currentLiabilities,1_bal_longTermDebt,1_bal_otherNonCurrentLiabilities,1_bal_nonCurrentDeferredLiabilities,1_bal_nonCurrentDeferredTaxesLiabilities,1_bal_longTermProvisions,1_bal_totalNonCurrentLiabilities,1_bal_retainedEarnings,1_bal_stockholdersEquity,1_bal_totalDebt,1_bal_totalAssets,1_bal_totalLiabilities,1_bal_workingCapital,1_bal_quickRatio,1_bal_currentRatio,1_bal_debtCapitalRatio,1_bal_debtAssetRatio,1_cas_depreciation,1_cas_stockBasedCompensation,1_cas_cashFlowOperatingActivities,1_cas_capitalExpenditure,1_cas_saleOfBusiness,1_cas_purchaseOfPPE,1_cas_cashFlowInvestingActivities,1_cas_cashDividendsPaid,1_cas_repurchaseOfCapitalStock,1_cas_cashFlowFinancingActivities,1_cas_freeCashFlow,2_inc_totalRevenue,2_inc_costOfRevenue,2_inc_grossProfit,2_inc_netIncome,2_inc_operatingIncome,2_inc_netIncomeContinuousOperations,2_inc_netInterestIncome,2_inc_interestIncome,2_inc_otherIncomeExpense,2_inc_operatingExpense,2_inc_totalExpenses,2_inc_taxProvision,2_inc_interestExpense,2_inc_SGA,2_inc_researchAndDevelopment,2_inc_ebit,2_inc_dilutedEPS,2_inc_basicEPS,2_bal_cashEquivalent,2_bal_receivables,2_bal_finishedGoods,2_bal_workInProcess,2_bal_rawMaterials,2_bal_otherCurrentAssets,2_bal_inventory,2_bal_currentAssets,2_bal_netPPE,2_bal_otherNonCurrentAssets,2_bal_financialAssets,2_bal_goodwill,2_bal_goodwillAndOtherIntangibleAssets,2_bal_otherIntangibleAssets,2_bal_nonCurrentAccountsReceivable,2_bal_totalNonCurrentAssets,2_bal_currentDebt,2_bal_payablesAndAccruedExpenses,2_bal_otherCurrentLiabilities,2_bal_currentLiabilities,2_bal_longTermDebt,2_bal_otherNonCurrentLiabilities,2_bal_nonCurrentDeferredLiabilities,2_bal_nonCurrent

### Split Data into Independent and Dependent Columns

In [22]:
X = df.drop('label',axis=1)
y = df['label']

In [23]:
print(X.shape,y.shape)

(728, 261) (728,)


### Train Test Split

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=66,shuffle=True,stratify=y)

### Handle Class Imbalance

In [25]:
def balance_dataset(X,y):

  df = pd.concat([X,y],axis=1)

  df_class_0 = df[df['label'] == 0]
  df_class_1 = df[df['label'] == 1]
  df_class_2 = df[df['label'] == 2]
  df_class_3 = df[df['label'] == 3]
  df_class_4 = df[df['label'] == 4]

  df_class_0_over = df_class_0.sample(252, replace=True)
  df_class_1_over = df_class_1.sample(252, replace=True)
  df_class_2_over = df_class_2.sample(252, replace=True)
  df_class_4_over = df_class_4.sample(252, replace=True)

  df_temp = pd.concat([df_class_0_over,df_class_1_over,df_class_2_over,df_class_3,df_class_4_over],axis=0)

  new_X = df_temp.drop('label',axis=1)
  new_y = df_temp['label']

  return new_X, new_y

### Cap Outliers

In [ ]:
def cap_ext_outliers(train_X,test_X):

  df_train = train_X.copy()
  df_test = test_X.copy()
    
  cols = df_train.columns
  cols = list(cols)

  for feature in cols:
    IQR = X_train[feature].quantile(0.75)- X_train[feature].quantile(0.25)
    ext_lower_bound = df_train[feature].quantile(0.75) - (IQR*3)
    ext_upper_bound = df_train[feature].quantile(0.75) + (IQR*3) 
      
    df_train[feature] = np.where(df_train[feature] < ext_lower_bound, ext_lower_bound, df_train[feature])
    df_train[feature] = np.where(df_train[feature] > ext_upper_bound, ext_upper_bound, df_train[feature])  

    df_test[feature] = np.where(df_test[feature] < ext_lower_bound, ext_lower_bound, df_test[feature])
    df_test[feature] = np.where(df_test[feature] > ext_upper_bound, ext_upper_bound, df_test[feature])  

  return df_train,df_test


In [46]:
def cap_outliers(train_X,test_X):

  df_train = train_X.copy()
  df_test = test_X.copy()
    
  cols = df_train.columns
  cols = list(cols)

  for feature in cols:
    IQR = X_train[feature].quantile(0.75)- X_train[feature].quantile(0.25)
    ext_lower_bound = df_train[feature].quantile(0.75) - (IQR*1.5)
    ext_upper_bound = df_train[feature].quantile(0.75) + (IQR*1.5) 
      
    df_train[feature] = np.where(df_train[feature] < ext_lower_bound, ext_lower_bound, df_train[feature])
    df_train[feature] = np.where(df_train[feature] > ext_upper_bound, ext_upper_bound, df_train[feature])  

    df_test[feature] = np.where(df_test[feature] < ext_lower_bound, ext_lower_bound, df_test[feature])
    df_test[feature] = np.where(df_test[feature] > ext_upper_bound, ext_upper_bound, df_test[feature])  

  return df_train,df_test

In [27]:
class cap_outlier_ext(BaseEstimator,TransformerMixin):
    
    def __init__(self, factor=3.0):
        self.factor = factor
        
    def outliers_iqr(self, X, y=None):
        X = pd.Series(X).copy()
        q1 = X.quantile(0.25)
        q3 = X.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (iqr * self.factor)
        upper_bound = q3 + (iqr * self.factor)
        # filter only those rows that are greater than lower_bound and less than upper_bound, 
        #i.e. drop values outside the given interval
        X.loc[X < lower_bound] = lower_bound
        X.loc[X > upper_bound] = upper_bound
        
        return pd.Series(X)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.apply(self.outliers_iqr)


In [48]:
class cap_outlier(BaseEstimator,TransformerMixin):
    
    def __init__(self, factor=1.5):
        self.factor = factor
        
    def outliers_iqr(self, X, y=None):
        X = pd.Series(X).copy()
        q1 = X.quantile(0.25)
        q3 = X.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (iqr * self.factor)
        upper_bound = q3 + (iqr * self.factor)
        # filter only those rows that are greater than lower_bound and less than upper_bound, 
        #i.e. drop values outside the given interval
        X.loc[X < lower_bound] = lower_bound
        X.loc[X > upper_bound] = upper_bound
        
        return pd.Series(X)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.apply(self.outliers_iqr)


In [50]:
# Cap outliers

l = X_train.shape[1]

ctrf_cap_outliers = ColumnTransformer(
                    [('capOutlier',cap_outlier(), slice(0,l))
                    ],remainder='passthrough') 


### Corelated Feaures

In [ ]:
# Select highly correlated features

def correlation(dataset, threshold):
  col_corr = set()
  corr_matrix = dataset.corr()
  for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i,j] > threshold:
          colname = corr_matrix.columns[i]
          col_corr.add(colname)
  return col_corr

In [ ]:
col_corr = correlation(X_train, 90)
col_corr

In [28]:
class removeCorrFeatures(BaseEstimator,TransformerMixin):
    
    def __init__(self, threshold):
        self.threshold = threshold
        self.correlated_features = None

    def fit(self, X, y=None):
        col_corr = set() 
        X = pd.DataFrame(X)
        corr_matrix = X.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if corr_matrix.iloc[i, j] > self.threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i]  # getting the name of column
                    col_corr.add(colname)
        self.correlated_features = col_corr
        return self

    def transform(self, X, y=None, **kwargs):
        return (pd.DataFrame(X)).drop(self.correlated_features, axis=1)


### Model Selection

In [31]:
def train_eval_model(model,train_X,train_y,test_X,test_y,use_sample_weights=False):

  if use_sample_weights:
    class_weights = class_weight.compute_sample_weight(
            class_weight='balanced',
            y=train_y)
    model.fit(train_X, train_y,sample_weight = class_weights)
  else:
    model.fit(train_X, train_y)  
      
  y_pred = model.predict(test_X)
  y_train_predict = model.predict(train_X)

  train_accuracy = round(accuracy_score(train_y, y_train_predict),3)
  test_accuracy = round(accuracy_score(test_y, y_pred),3)

  train_f1 = round(f1_score(train_y, y_train_predict,zero_division=np.nan, average='weighted'),3)
  test_f1 = round(f1_score(test_y, y_pred,zero_division=np.nan, average='weighted'),3)
    
  print('Train accuracy',train_accuracy)
  print('Test accuracy',test_accuracy) 
  print
    
  print('Train F1 Score',train_f1)
  print('Test F1 Score',test_f1) 
  print("")

  cross_val_accuracy = round(np.mean(cross_val_score(model,train_X,train_y,scoring='accuracy',cv=5)),3)
 
  cross_val_f1 = round(np.mean(cross_val_score(model,train_X,train_y,
                                         scoring=make_scorer(f1_score, average='weighted'),
                                         cv=5)),3)
  
  print('Cross val accuracy:',cross_val_accuracy)
  print('Cross val f1:',cross_val_f1)
  print("")
  print(pd.crosstab(test_y,y_pred))
  print("")
  print(classification_report(test_y, y_pred,zero_division=np.nan))


## Baseline Evaluation 

Default parameters, no modification to dataset.

In [ ]:
# Drop Industry

X_train = X_train.drop('industry',axis=1)
X_test = X_test.drop('industry',axis=1)

### XGBoost

In [ ]:
model = xgb.XGBClassifier(
                       random_state=0)      

train_eval_model(model, X_train, y_train, X_test, y_test)


In [ ]:
model = xgb.XGBClassifier(
                       random_state=50)      

train_eval_model(model, X_train, y_train, X_test, y_test,use_sample_weights=True)

### Adaboost

In [ ]:
model = AdaBoostClassifier(n_estimators=200,algorithm='SAMME')
train_eval_model(model, X_train, y_train, X_test, y_test)

In [ ]:
model = AdaBoostClassifier(n_estimators=200,algorithm='SAMME.R')
train_eval_model(model, X_train, y_train, X_test, y_test)

### Random Forest

In [ ]:
model = RandomForestClassifier(random_state=0)
train_eval_model(model, X_train, y_train, X_test, y_test)

### Gradient Boosting

In [ ]:
model = RandomForestClassifier(random_state=0)
train_eval_model(model, X_train, y_train, X_test, y_test,use_sample_weights=True)

### Hyperparameter Tuning

In [ ]:
X_train, y_train = balance_dataset(X_train, y_train)
X_train.shape

In [ ]:
counter = Counter(y_train)
for k, v in counter.items():
  dist = v / len(y_train) * 100
  print(f"Class={k}, n={v} ({dist}%)")

In [ ]:
ctrf_cap_outliers

### XGBoost

In [ ]:
clf = xgb.XGBClassifier(random_state=0)     

pipe = Pipeline(
    steps=[("capOutliers",ctrf_cap_outliers), 
           ("removeCorrFeatures",removeCorrFeatures(threshold=0.90)),
           ("select", SelectKBest(score_func=mutual_info_classif, k=100)),    
           ("classifier",clf)])    

param_grid = {
    "classifier__max_depth": [2,3,4,5,6,7],
    "classifier__n_estimators":[50,70,90,110,150,200,250,300,250,400],
    "classifier__learning_rate":[0.05,0.07,0.09,0.11,0.15],
    "classifier__gamma":[0.01,0.1,1,2,3,4,5,6,7]
}
           
gridSearch = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='f1_macro',
    verbose=10,
    n_jobs=-1,
    cv=5
)

gridSearch = GridSearchCV(pipe,param_grid=param_grid, n_jobs=-1)
gridSearch.fit(X_train,y_train)

In [ ]:
gridSearch.best_score_

In [ ]:
gridSearch.best_params_

In [ ]:
clf = xgb.XGBClassifier(random_state=50,
                          max_depth = 7,
                          learning_rate = 0.15,
                          gamma = 0.1,
                          n_estimators = 70)   

pipe = Pipeline(
    steps=[("capOutliers",ctrf_cap_outliers), 
           ("removeCorrFeatures",removeCorrFeatures(threshold=0.90)),
           ("select", SelectKBest(score_func=mutual_info_classif, k=100)),    
           ("classifier",clf)])    

train_eval_model(model, X_train, y_train, X_test, y_test)

### Random Forest

In [ ]:
clf=RandomForestClassifier(random_state=0)

pipe = Pipeline(
    steps=[("capOutliers",ctrf_cap_outliers), 
           ("removeCorrFeatures",removeCorrFeatures(threshold=0.90)),
           ("select", SelectKBest(score_func=mutual_info_classif, k=100)),    
           ("classifier",clf)])    

param_grid = {
    "classifier__criterion":["gini","entropy"],
    "classifier__max_depth": [2,3,4,5,6],
    "classifier__n_estimators":[50,70,90,110,150,200,250,300,250,400],
}
           
gridSearch = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='f1_macro',
    verbose=10,
    n_jobs=-1,
    cv=5
)

gridSearch = GridSearchCV(pipe,param_grid=param_grid, n_jobs=-1)
gridSearch.fit(X_train,y_train)

In [ ]:
gridSearch.best_score_

In [ ]:
gridSearch.best_params_

In [ ]:
clf=RandomForestClassifier(random_state=0,
                           max_depth = 6,
                           criterion = 'entropy',
                           n_estimators = 300)

pipe = Pipeline(
    steps=[("capOutliers",ctrf_cap_outliers),
           ("removeCorrFeatures",removeCorrFeatures(threshold=0.90)),
           ("select", SelectKBest(score_func=mutual_info_classif, k=100)),    
           ("classifier", RandomForestClassifier()),]
)
train_eval_model(pipe, X_train, y_train, X_test, y_test)

### XGBoost

In [ ]:

pipe = Pipeline(
    steps=[("capOutliers",ctrf_cap_outliers),
           ("removeCorrFeatures",removeCorrFeatures(threshold=0.90)),
           ("select", SelectKBest(score_func=mutual_info_classif, k=100)),    
           ("classifier",  xgb.XGBClassifier()),]
)
train_eval_model(pipe, X_train, y_train, X_test, y_test)

## Optimised Models

In [66]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=66,shuffle=True,stratify=y)

In [67]:
# Handle Imblanced Dataset 

# Resampling doesn't improve performance so comment out.

#X_train, y_train = balance_dataset(X_train,y_train)

In [68]:
# Drop Industry

X_train = X_train.drop('industry',axis=1)
X_test = X_test.drop('industry',axis=1)

### XGBoost

In [69]:
clf = xgb.XGBClassifier(
                       max_depth = 3,
                       n_estimators = 120,
                       learning_rate = 0.14,
                       gamma = 0.01,
                       reg_lambda = 0.5,
                       reg_alpha = 0.6,
                       min_child_weight=5,
                       random_state=100)      

pipe = Pipeline(
    steps=[("capOutliers",ctrf_cap_outliers),
           ("removeCorrFeatures",removeCorrFeatures(threshold=0.90)),
           ("classifier", clf),]
)

train_eval_model(pipe, X_train, y_train, X_test, y_test)

Train accuracy 1.0
Test accuracy 0.712
Train F1 Score 1.0
Test F1 Score 0.705

Cross val accuracy: 0.637
Cross val f1: 0.624

col_0  0   1   2   3  4
label                  
0      4   1   0   0  0
1      1  12   4   0  0
2      0   2  37  16  0
3      0   0  12  50  1
4      0   0   0   5  1

              precision    recall  f1-score   support

           0       0.80      0.80      0.80         5
           1       0.80      0.71      0.75        17
           2       0.70      0.67      0.69        55
           3       0.70      0.79      0.75        63
           4       0.50      0.17      0.25         6

    accuracy                           0.71       146
   macro avg       0.70      0.63      0.65       146
weighted avg       0.71      0.71      0.71       146



### Gradient Boost

In [70]:
clf=GradientBoostingClassifier( n_estimators = 75,
                                  max_depth = 3,
                                  learning_rate = 0.16 )


pipe = Pipeline(
    steps=[("capOutliers",ctrf_cap_outliers),
           ("removeCorrFeatures",removeCorrFeatures(threshold=0.90)),
           ("classifier", clf),]
)

train_eval_model(pipe, X_train, y_train, X_test, y_test)

Train accuracy 1.0
Test accuracy 0.712
Train F1 Score 1.0
Test F1 Score 0.728

Cross val accuracy: 0.653
Cross val f1: 0.639

col_0  0   1   2   3  4
label                  
0      5   0   0   0  0
1      1  10   6   0  0
2      1   1  38  15  0
3      1   0  10  51  1
4      0   0   0   6  0

              precision    recall  f1-score   support

           0       0.62      1.00      0.77         5
           1       0.91      0.59      0.71        17
           2       0.70      0.69      0.70        55
           3       0.71      0.81      0.76        63
           4       0.00      0.00       nan         6

    accuracy                           0.71       146
   macro avg       0.59      0.62      0.73       146
weighted avg       0.70      0.71      0.73       146



### Random Forest

In [71]:
clf=RandomForestClassifier(n_estimators = 100,
                             max_depth = 6,
                             random_state=0)

pipe = Pipeline(
    steps=[("capOutliers",ctrf_cap_outliers),
           ("removeCorrFeatures",removeCorrFeatures(threshold=0.90)),
           ("classifier", clf),]
)

train_eval_model(pipe, X_train, y_train, X_test, y_test)

Train accuracy 0.875
Test accuracy 0.705
Train F1 Score 0.859
Test F1 Score 0.713

Cross val accuracy: 0.655
Cross val f1: 0.635

col_0  0  1   2   3
label              
0      5  0   0   0
1      2  9   6   0
2      0  2  34  19
3      0  0   8  55
4      0  0   0   6

              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       0.82      0.53      0.64        17
           2       0.71      0.62      0.66        55
           3       0.69      0.87      0.77        63
           4        nan      0.00       nan         6

    accuracy                           0.71       146
   macro avg       0.73      0.60      0.73       146
weighted avg       0.71      0.71      0.71       146



Need to do more work to deal with bias-variance problem. Tried early stopping but that didn't make much difference
to performance. Feature selection doesn't improve performance, so it was removed.